In [1]:
# Import the necessary libraries
from PIL import Image
from numpy import asarray
import numpy as np
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from keras.utils import np_utils
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
print('all imported')

all imported


In [2]:
def readImage(filePath):
    img = Image.open(filePath)
    
    size=(30,40)
    #resize image
    out = img.resize(size)

    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(out)
    numpydata = np.repeat(numpydata[:, :, np.newaxis], 3, axis=2)
    # <class 'numpy.ndarray'>
    #print(type(numpydata))

    #  shape
    #print(numpydata.shape)
    return numpydata
print(readImage('data/nervus/ISIC_0001769.jpg'))

[[[[142 112  87]
   [142 112  87]
   [142 112  87]]

  [[157 121  91]
   [157 121  91]
   [157 121  91]]

  [[168 130  98]
   [168 130  98]
   [168 130  98]]

  ...

  [[174 139 113]
   [174 139 113]
   [174 139 113]]

  [[169 135 110]
   [169 135 110]
   [169 135 110]]

  [[159 130 108]
   [159 130 108]
   [159 130 108]]]


 [[[148 115  89]
   [148 115  89]
   [148 115  89]]

  [[161 123  95]
   [161 123  95]
   [161 123  95]]

  [[170 133 103]
   [170 133 103]
   [170 133 103]]

  ...

  [[175 139 114]
   [175 139 114]
   [175 139 114]]

  [[171 136 112]
   [171 136 112]
   [171 136 112]]

  [[164 132 111]
   [164 132 111]
   [164 132 111]]]


 [[[155 120  95]
   [155 120  95]
   [155 120  95]]

  [[165 129 102]
   [165 129 102]
   [165 129 102]]

  [[170 133 103]
   [170 133 103]
   [170 133 103]]

  ...

  [[178 144 120]
   [178 144 120]
   [178 144 120]]

  [[173 140 116]
   [173 140 116]
   [173 140 116]]

  [[168 136 115]
   [168 136 115]
   [168 136 115]]]


 ...


 [[[154 129 

In [3]:
listFiles = os.listdir('data/melanoma/')
print(listFiles[:5])

['ISIC_0012099.jpg', 'ISIC_0012151.jpg', 'ISIC_0012288.jpg', 'ISIC_0012434.jpg', 'ISIC_0013232.jpg']


In [4]:
#load Benign
listCancer = []
listResults = []
for elem in listFiles:
    img = readImage('data/melanoma/'+elem)
    listCancer += [img]
    listResults += [np.array([1])]
print(listCancer[:3])
print(img.shape)

[array([[[[ 54,  55,  58],
         [ 54,  55,  58],
         [ 54,  55,  58]],

        [[168, 168, 169],
         [168, 168, 169],
         [168, 168, 169]],

        [[184, 184, 185],
         [184, 184, 185],
         [184, 184, 185]],

        ...,

        [[190, 180, 183],
         [190, 180, 183],
         [190, 180, 183]],

        [[172, 164, 169],
         [172, 164, 169],
         [172, 164, 169]],

        [[ 66,  72,  89],
         [ 66,  72,  89],
         [ 66,  72,  89]]],


       [[[ 99,  99, 102],
         [ 99,  99, 102],
         [ 99,  99, 102]],

        [[179, 178, 179],
         [179, 178, 179],
         [179, 178, 179]],

        [[185, 183, 184],
         [185, 183, 184],
         [185, 183, 184]],

        ...,

        [[192, 180, 184],
         [192, 180, 184],
         [192, 180, 184]],

        [[184, 172, 175],
         [184, 172, 175],
         [184, 172, 175]],

        [[100,  97, 108],
         [100,  97, 108],
         [100,  97, 108]]],


       

In [5]:
listFiles = os.listdir('data/nervus/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/nervus/'+elem)
    listResults += [np.array([0])]
    listCancer += [img]

['ISIC_0001769.jpg', 'ISIC_0001852.jpg', 'ISIC_0001871.jpg', 'ISIC_0003462.jpg', 'ISIC_0003539.jpg']


In [6]:
listFiles = os.listdir('data/seborrheic_keratosis/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/seborrheic_keratosis/'+elem)
    listResults += [np.array([2])]
    listCancer += [img]

['ISIC_0012143.jpg', 'ISIC_0012204.jpg', 'ISIC_0012210.jpg', 'ISIC_0012254.jpg', 'ISIC_0012380.jpg']


In [7]:
print(len(listCancer))

150


In [8]:
# Setting LR for different number of Epochs
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    lr *= float(epoch) / 100
    print('Learning rate: ', epoch / 300)
    return lr


In [9]:
# Basic ResNet Building Block
def resnet_layer(inputs, conv_first = False,
				num_filters = 16,
				kernel_size = 3,
				strides = 1,
				activation ='relu',
				batch_normalization = True):
	conv = Conv2D(num_filters,
				kernel_size = kernel_size,
				strides = strides,
				padding ='same',
				kernel_initializer ='he_normal',
				kernel_regularizer = l2(1e-4))

	x = inputs
	if conv_first:
		x = conv(x)
		if batch_normalization:
			x = BatchNormalization()(x)
		if activation is not None:
			x = Activation(activation)(x)
	else:
		if batch_normalization:
			x = BatchNormalization()(x)
		if activation is not None:
			x = Activation(activation)(x)
		x = conv(x)
	return x

In [10]:
# ResNet V2 architecture
def resnet_v2(input_shape, depth, num_classes = 10):
	if (depth - 2) % 9 != 0:
		raise ValueError('depth should be 9n + 2 (eg 56 or 110 in [b])')
	# Start model definition.
	num_filters_in = 16
	num_res_blocks = int((depth - 2) / 9)

	inputs = Input(shape = input_shape)
	# v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
	x = resnet_layer(inputs = inputs,num_filters = num_filters_in,conv_first = True)

	# Instantiate the stack of residual units
	for stage in range(3):
		for res_block in range(num_res_blocks):
			activation = 'relu'
			batch_normalization = True
			strides = 1
			if stage == 0:
				num_filters_out = num_filters_in * 4
				if res_block == 0: # first layer and first stage
					activation = None
					batch_normalization = False
			else:
				num_filters_out = num_filters_in * 2
				if res_block == 0: # first layer but not first stage
					strides = 2 # downsample

			# bottleneck residual unit
			y = resnet_layer(inputs = x,
							num_filters = num_filters_in,
							kernel_size = 1,
							strides = strides,
							activation = activation,
							batch_normalization = batch_normalization)
			y = resnet_layer(inputs = y,
							num_filters = num_filters_in,
							conv_first = False)
			y = resnet_layer(inputs = y,
							num_filters = num_filters_out,
							kernel_size = 1,
							conv_first = False)
			if res_block == 0:
				# linear projection residual shortcut connection to match
				# changed dims
				x = resnet_layer(inputs = x,
								num_filters = num_filters_out,
								kernel_size = 1,
								strides = strides,
								activation = None,
								batch_normalization = False)
			x = keras.layers.add([x, y])

		num_filters_in = num_filters_out

	# Add classifier on top.
	# v2 has BN-ReLU before Pooling
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = AveragePooling2D(pool_size = 8)(x)
	y = Flatten()(x)
	outputs = Dense(num_classes,
					activation ='softmax',
					kernel_initializer ='he_normal')(y)

	# Instantiate model.
	model = Model(inputs = inputs, outputs = outputs)
	return model

In [16]:
x_train=[]
y_train=[]
accs=[]
def main(pz):
    
    per = np.random.permutation(len(listCancer))
    ln = int(len(listCancer) * 0.6)
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    positions = []
    #for i in range(ln):
    #    positions += [per[i]]
    val = 1
    while ln > 0:
        val = 1+val
        if val == 3:
            val=0
        ln-=1
        for i in range(len(per)):
            if per[i] in positions:
                continue
            if listResults[per[i]] == np.array([val]):
                positions += [per[i]]
                break
    for i in range(len(listCancer)):
        if i in positions:
            x_train += [listCancer[i]]
            y_train += [listResults[i]]
        else:
            x_test += [listCancer[i]]
            y_test += [listResults[i]]
            
    ln = len(x_test) // 2
    
    x_valid = x_test[ln:]
    y_valid = y_test[ln:]
    
    x_test = x_test[:ln]
    y_test = y_test[:ln]
    
    
    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 40, 30, 9)
    y_train = np.array(y_train)
    

    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 40, 30, 9)
    y_test = np.array(y_test)
    
    x_valid = np.array(x_valid)
    x_valid = x_valid.reshape(len(x_valid), 40, 30, 9)
    y_valid = np.array(y_valid)
    
    
    batch_size = 32 # original ResNet paper uses batch_size = 128 for training
    epochs = 10
    data_augmentation = True
    num_classes = 10

    # Data Preprocessing
    subtract_pixel_mean = True
    n = 3

    # Select ResNet Version
    version = 2

    # Computed depth of
    if version == 1:
        depth = n * 6 + 2
    elif version == 2:
        depth = n * 9 + 2

    # Model name, depth and version
    model_type = 'ResNet % dv % d' % (depth, version)

    # use the data
    print(x_train[:3])
    print(y_train[:3])
    print(type(x_train[0]))
    print(type(x_train))
    print(type(y_train[0]))
    print(type(y_train))

    # Input image dimensions.
    input_shape = x_train.shape[1:]
    print(input_shape)

    # Normalize data.
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    x_valid = x_test.astype('float32') / 255

    # If subtract pixel mean is enabled
    if subtract_pixel_mean:
        x_train_mean = np.mean(x_train, axis = 0)
        x_train -= x_train_mean
        x_test -= x_train_mean

    # Print Training and Test Samples
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    # Convert class vectors to binary class matrices.
    y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)
    y_valid = keras.utils.np_utils.to_categorical(y_valid, num_classes)
    
    
    model = resnet_v2(input_shape = input_shape, depth = depth)

    model.compile(loss ='categorical_crossentropy',
                optimizer = Adam(learning_rate = lr_schedule(0)),
                metrics =['accuracy'])
    model.summary()
    print(model_type)

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    model_name = 'cifar10_% s_model.{epoch:03d}.h5' % model_type
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filepath = os.path.join(save_dir, model_name)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath = filepath,
                                monitor ='val_acc',
                                verbose = 1,
                                save_best_only = True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1),
                                cooldown = 0,
                                patience = 5,
                                min_lr = 0.5e-6)

    callbacks = [checkpoint, lr_reducer, lr_scheduler]

    # Run training, with or without data augmentation.
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                batch_size = batch_size,
                epochs = epochs,
                validation_data =(x_test, y_test),
                shuffle = True,
                callbacks = callbacks)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            # set input mean to 0 over the dataset
            featurewise_center = False,
            # set each sample mean to 0
            samplewise_center = False,
            # divide inputs by std of dataset
            featurewise_std_normalization = False,
            # divide each input by its std
            samplewise_std_normalization = False,
            # apply ZCA whitening
            zca_whitening = False,
            # epsilon for ZCA whitening
            zca_epsilon = 1e-06,
            # randomly rotate images in the range (deg 0 to 180)
            rotation_range = 0,
            # randomly shift images horizontally
            width_shift_range = 0.1,
            # randomly shift images vertically
            height_shift_range = 0.1,
            # set range for random shear
            shear_range = 0.,
            # set range for random zoom
            zoom_range = 0.,
            # set range for random channel shifts
            channel_shift_range = 0.,
            # set mode for filling points outside the input boundaries
            fill_mode ='nearest',
            # value used for fill_mode = "constant"
            cval = 0.,
            # randomly flip images
            horizontal_flip = True,
            # randomly flip images
            vertical_flip = False,
            # set rescaling factor (applied before any other transformation)
            rescale = None,
            # set function that will be applied on each input
            preprocessing_function = None,
            # image data format, either "channels_first" or "channels_last"
            data_format = None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split = 0.1)

        # Compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        print('------------------')
        print(x_train)
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size),
                            validation_data =(x_valid, y_valid),
                            epochs = 100, verbose = 1, workers = 4,
                            callbacks = callbacks)

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose = 1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    ret = model.predict(x_test)
    result = []
    pred = []
    for a,b in zip(ret, y_test):
        pred+=[a.argmax(axis=-1)]
        result+=[b.argmax(axis=-1)]
    print(pred)
    print(result)
    precision = precision_score(result, pred, average='weighted')
    recall = recall_score(result, pred, average='weighted')
     # calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
    fmeasure = f1_score(result, pred, average='weighted')
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1 Score: ', fmeasure)
    #precisions += [precision]
    #recalls += [recall]
    if scores[1] > 0.5:
        model.save('saved_models/skinResNetV'+str(pz)+'.h5')
    return [scores[1], precision, recall, fmeasure]

In [17]:
accs=[]
precisions = []
recalls = []
fmeasures = []
for i in range(10):
    ret = main(i)
    accs+=[ret[0]]
    precisions+=[ret[1]]
    recalls+=[ret[2]]
    fmeasures+=[ret[3]]
print(accs)
print(precisions)
print(recalls)
print(fmeasures)

[[[[156 130 112 ... 156 130 112]
   [155 132 114 ... 155 132 114]
   [162 144 131 ... 162 144 131]
   ...
   [173 152 138 ... 173 152 138]
   [177 161 153 ... 177 161 153]
   [188 176 178 ... 188 176 178]]

  [[154 127 107 ... 154 127 107]
   [158 136 121 ... 158 136 121]
   [162 142 126 ... 162 142 126]
   ...
   [174 153 142 ... 174 153 142]
   [175 153 140 ... 175 153 140]
   [189 177 179 ... 189 177 179]]

  [[145 112  88 ... 145 112  88]
   [161 142 129 ... 161 142 129]
   [158 132 110 ... 158 132 110]
   ...
   [175 155 147 ... 175 155 147]
   [168 139 119 ... 168 139 119]
   [181 162 155 ... 181 162 155]]

  ...

  [[199 191 190 ... 199 191 190]
   [191 179 172 ... 191 179 172]
   [173 149 129 ... 173 149 129]
   ...
   [181 160 145 ... 181 160 145]
   [181 164 150 ... 181 164 150]
   [177 158 146 ... 177 158 146]]

  [[192 179 175 ... 192 179 175]
   [197 189 186 ... 197 189 186]
   [173 149 127 ... 173 149 127]
   ...
   [180 160 144 ... 180 160 144]
   [179 160 144 ... 179 16

5/5 [==============================] - 11s 1s/step - loss: 2.7383 - accuracy: 0.1286 - val_loss: 2.8878 - val_accuracy: 0.0000e+00
Epoch 2/100
Learning rate:  0.0033333333333333335
5/5 [==============================] - 6s 1s/step - loss: 2.7462 - accuracy: 0.1357 - val_loss: 2.8791 - val_accuracy: 0.0000e+00
Epoch 3/100
Learning rate:  0.006666666666666667
5/5 [==============================] - 4s 854ms/step - loss: 2.7055 - accuracy: 0.1571 - val_loss: 2.8742 - val_accuracy: 0.4043
Epoch 4/100
Learning rate:  0.01
5/5 [==============================] - 5s 1s/step - loss: 2.6679 - accuracy: 0.1714 - val_loss: 2.8660 - val_accuracy: 0.4043
Epoch 5/100
Learning rate:  0.013333333333333334
5/5 [==============================] - 5s 1s/step - loss: 2.6235 - accuracy: 0.2357 - val_loss: 2.8564 - val_accuracy: 0.4043
Epoch 6/100
Learning rate:  0.016666666666666666
5/5 [==============================] - 4s 789ms/step - loss: 2.5600 - accuracy: 0.3071 - val_loss: 2.8552 - val_accuracy: 0.4043

5/5 [==============================] - 4s 738ms/step - loss: 1.2367 - accuracy: 0.7571 - val_loss: 4.5669 - val_accuracy: 0.0000e+00
Epoch 34/100
Learning rate:  0.11
5/5 [==============================] - 4s 741ms/step - loss: 1.2403 - accuracy: 0.7571 - val_loss: 4.4370 - val_accuracy: 0.0000e+00
Epoch 35/100
Learning rate:  0.11333333333333333
5/5 [==============================] - 4s 744ms/step - loss: 1.1666 - accuracy: 0.8071 - val_loss: 5.0076 - val_accuracy: 0.0000e+00
Epoch 36/100
Learning rate:  0.11666666666666667
5/5 [==============================] - 4s 751ms/step - loss: 1.1534 - accuracy: 0.7786 - val_loss: 4.8112 - val_accuracy: 0.0000e+00
Epoch 37/100
Learning rate:  0.12
5/5 [==============================] - 4s 741ms/step - loss: 1.1478 - accuracy: 0.7929 - val_loss: 3.3346 - val_accuracy: 0.0000e+00
Epoch 38/100
Learning rate:  0.12333333333333334
5/5 [==============================] - 4s 871ms/step - loss: 1.2032 - accuracy: 0.7857 - val_loss: 3.2537 - val_accuracy

5/5 [==============================] - 4s 835ms/step - loss: 1.0313 - accuracy: 0.8143 - val_loss: 7.8269 - val_accuracy: 0.0000e+00
Epoch 66/100
Learning rate:  0.21666666666666667
5/5 [==============================] - 4s 722ms/step - loss: 1.0003 - accuracy: 0.8429 - val_loss: 8.1524 - val_accuracy: 0.0000e+00
Epoch 67/100
Learning rate:  0.22
5/5 [==============================] - 4s 729ms/step - loss: 0.9493 - accuracy: 0.8857 - val_loss: 6.7237 - val_accuracy: 0.0000e+00
Epoch 68/100
Learning rate:  0.22333333333333333
5/5 [==============================] - 4s 734ms/step - loss: 0.9213 - accuracy: 0.8571 - val_loss: 7.6688 - val_accuracy: 0.0000e+00
Epoch 69/100
Learning rate:  0.22666666666666666
5/5 [==============================] - 4s 740ms/step - loss: 0.9299 - accuracy: 0.8429 - val_loss: 9.9127 - val_accuracy: 0.0000e+00
Epoch 70/100
Learning rate:  0.23
5/5 [==============================] - 4s 773ms/step - loss: 0.9687 - accuracy: 0.8786 - val_loss: 9.6624 - val_accuracy

5/5 [==============================] - 4s 708ms/step - loss: 0.7970 - accuracy: 0.9143 - val_loss: 10.3109 - val_accuracy: 0.0000e+00
Epoch 98/100
Learning rate:  0.3233333333333333
5/5 [==============================] - 4s 651ms/step - loss: 0.8297 - accuracy: 0.9000 - val_loss: 10.4458 - val_accuracy: 0.0000e+00
Epoch 99/100
Learning rate:  0.32666666666666666
5/5 [==============================] - 4s 662ms/step - loss: 0.7787 - accuracy: 0.9214 - val_loss: 10.1900 - val_accuracy: 0.0000e+00
Epoch 100/100
Learning rate:  0.33
2/2 [==============================] - 0s 83ms/step - loss: 2.0352 - accuracy: 0.4468
Test loss: 2.0352301597595215
Test accuracy: 0.44680851697921753
[2, 1, 1, 2, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 2, 2, 2, 2, 0, 0, 2, 1, 1, 0, 2, 2, 0, 0, 2, 2, 0, 2, 0, 1, 1, 2, 1, 0, 2, 0, 2, 2, 0, 1, 0, 0, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2]
Precision:  0.6534000834376303


...

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


[[[[156 130 112 ... 156 130 112]
   [155 132 114 ... 155 132 114]
   [162 144 131 ... 162 144 131]
   ...
   [173 152 138 ... 173 152 138]
   [177 161 153 ... 177 161 153]
   [188 176 178 ... 188 176 178]]

  [[154 127 107 ... 154 127 107]
   [158 136 121 ... 158 136 121]
   [162 142 126 ... 162 142 126]
   ...
   [174 153 142 ... 174 153 142]
   [175 153 140 ... 175 153 140]
   [189 177 179 ... 189 177 179]]

  [[145 112  88 ... 145 112  88]
   [161 142 129 ... 161 142 129]
   [158 132 110 ... 158 132 110]
   ...
   [175 155 147 ... 175 155 147]
   [168 139 119 ... 168 139 119]
   [181 162 155 ... 181 162 155]]

  ...

  [[199 191 190 ... 199 191 190]
   [191 179 172 ... 191 179 172]
   [173 149 129 ... 173 149 129]
   ...
   [181 160 145 ... 181 160 145]
   [181 164 150 ... 181 164 150]
   [177 158 146 ... 177 158 146]]

  [[192 179 175 ... 192 179 175]
   [197 189 186 ... 197 189 186]
   [173 149 127 ... 173 149 127]
   ...
   [180 160 144 ... 180 160 144]
   [179 160 144 ... 179 16

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\image_data_generator.py:938: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (140, 40, 30, 9) (9 channels).
  warnings.warn(
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (140, 40, 30, 9) (9 channels).
  warnings.warn('NumpyArrayIterator is set to use the '
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and w

5/5 [==============================] - 9s 991ms/step - loss: 2.7088 - accuracy: 0.1714 - val_loss: 2.8548 - val_accuracy: 0.0000e+00
Epoch 2/100
Learning rate:  0.0033333333333333335
5/5 [==============================] - 4s 686ms/step - loss: 2.7140 - accuracy: 0.2000 - val_loss: 2.8201 - val_accuracy: 0.0000e+00
Epoch 3/100
Learning rate:  0.006666666666666667
5/5 [==============================] - 4s 721ms/step - loss: 2.6894 - accuracy: 0.1857 - val_loss: 2.7937 - val_accuracy: 0.0000e+00
Epoch 4/100
Learning rate:  0.01
5/5 [==============================] - 4s 693ms/step - loss: 2.6277 - accuracy: 0.2143 - val_loss: 2.7728 - val_accuracy: 0.0000e+00
Epoch 5/100
Learning rate:  0.013333333333333334
5/5 [==============================] - 4s 802ms/step - loss: 2.5539 - accuracy: 0.2786 - val_loss: 2.7570 - val_accuracy: 0.0000e+00
Epoch 6/100
Learning rate:  0.016666666666666666


KeyboardInterrupt: 

In [ ]:
print(len(accs))
print('accs: ', accs)
print('precisions: ', precisions)
print('recalls: ', recalls)
print('fmeasures: ', fmeasures)

In [ ]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

In [1]:
accs=  [0.3333333432674408, 0.5666666626930237, 0.5666666626930237, 0.4333333373069763, 0.4000000059604645, 0.46666666865348816, 0.4333333373069763, 0.46666666865348816, 0.5666666626930237, 0.6333333253860474]
precisions=  [0.4483333333333333, 0.736904761904762, 0.6722222222222223, 0.43333333333333335, 0.7, 0.6844444444444444, 0.7225108225108224, 0.6288888888888889, 0.6428571428571428, 0.7252723311546841]
recalls=  [0.3333333333333333, 0.5666666666666667, 0.5666666666666667, 0.43333333333333335, 0.4, 0.4666666666666667, 0.43333333333333335, 0.4666666666666667, 0.5666666666666667, 0.6333333333333333]
fmeasures=  [0.36973039215686276, 0.637989417989418, 0.6140056022408964, 0.43333333333333335, 0.5066666666666666, 0.5473039215686274, 0.5414814814814815, 0.5333333333333333, 0.5830429732868756, 0.6759259259259259]

In [2]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = precisions

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = recalls

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = fmeasures

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

(0.4199757689864667, 0.5533575659363238)
(0.5605134999069246, 0.7184399562230022)
(0.4199757642166506, 0.5533575691166828)
(0.47819747292875353, 0.6103651366679308)


In [13]:
from imblearn.over_sampling import SMOTE
print(len(listCancer))
print(len(listResults))
oversample = SMOTE()
X, y = oversample.fit_resample(np.array(listCancer).reshape(len(listCancer),-1), listResults)
print(len(X))
print(len(y))
listCancer = list(X)
listResults = list(y)
print(listResults)

150
150
234
234
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
